## Adding Spatial Metadata to AORC Forcing

**Authors**: Tony Castronova <acastronova@cuahsi.org>, Irene Garousi-Nejad <igarousi@cuahsi.org>  
**Last Updated**: 03.31.2023

**Description**:  

This notebook demonstrates how to add spatial metadata to the AORC v1.0 forcing data that is stored on HydroShare's THREDDs. The original AORC v1.0 data contains `east_west` and `south_north` which allows us to slice the gridded data via `x` and `y` indices. It is necessary to add additional spatially-related metadata (e.g. coordinate reference system) to enable spatial querying and visualization of these data. This notebook demonstrates one method for doing this.

**Software Requirements**

This notebook was developed using the following software and operating system versions.

OS: MacOS Ventura 13.0.1  
Python: 3.10.0
Zarr: 2.13.2  
NetCDF4: 1.6.1  
xarray: 0.17.0  
fsspec: 0.8.7  
dask: 2021.3.0  
numpy: 1.24.1
rioxarray: 0.13.3

---

In [30]:
import re
import numpy
import xarray
import rioxarray 
import matplotlib.pyplot as plt

Load the AORC v1.0 data via HydroShare's THREDDS

In [29]:
# load a single month of data
ds_aorc = xarray.open_dataset('http://thredds.hydroshare.org/thredds/dodsC/aorc/data/16/201001.nc',
                              chunks={'Time': 10, 'west_east': 285, 'south_north':275},
                              decode_coords="all" )

Notice that the `south_north` and `west_east` dimensions contain indices and there do not exist coordinates containing values for these dimensions.

In [7]:
ds_aorc

<xarray.Dataset>
Dimensions:     (Time: 744, south_north: 820, west_east: 855)
Dimensions without coordinates: Time, south_north, west_east
Data variables:
    LWDOWN      (Time, south_north, west_east) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    PSFC        (Time, south_north, west_east) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    Q2D         (Time, south_north, west_east) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    RAINRATE    (Time, south_north, west_east) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    SWDOWN      (Time, south_north, west_east) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    T2D         (Time, south_north, west_east) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    Times       (Time) |S64 dask.array<chunksize=(10,), meta=np.ndarray>
    U2D         (Time, south_north, west_east) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    V2D         (Time, south_north, west_east) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    valid_time  (Time) datetime64[ns] dask.array<chunksize=(10,), meta=np.ndarray>
Attributes:
    history:       Fri Dec 16 20:05:28 2022: ncks /job/data/2010/2010013000.L...
    NCO:           4.7.2
    DODS.strlen:   20
    DODS.dimName:  DateStrLen

In [8]:
ds_aorc.south_north

<xarray.DataArray 'south_north' (south_north: 820)>
array([  0,   1,   2, ..., 817, 818, 819])
Dimensions without coordinates: south_north

Load the GeoSpatial Metadata for NWM v2.0 that is stored in HydroShare. The `WRF_Hydro_NWM_geospatial_data_template_land_GIS.nc` file is part of the NWM v2.0 domain files and contains spatial metadata that we can add to the AORC dataset. We can access this via HydroShare's THREDDS too.

https://www.hydroshare.org/resource/2a8a3566e1c84b8eb3871f30841a3855/

In [11]:
ds_meta = xarray.open_dataset('http://thredds.hydroshare.org/thredds/dodsC/hydroshare/resources/2a8a3566e1c84b8eb3871f30841a3855/data/contents/WRF_Hydro_NWM_geospatial_data_template_land_GIS.nc')
ds_meta

<xarray.Dataset>
Dimensions:  (y: 3840, x: 4608)
Coordinates:
  * y        (y) float64 1.919e+06 1.918e+06 1.917e+06 ... -1.919e+06 -1.92e+06
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
Data variables:
    crs      |S64 ...
Attributes:
    proj4:             +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30...
    processing_notes:  Created: Thu Aug 02 19:38:45 2018
    Conventions:       CF-1.5
    GDAL_DataType:     Generic
    Source_Software:   WRF-Hydro GIS Pre-processor v5 (03/2018)
    history:           Created Thu Aug 02 19:39:09 2018
    DODS.strlen:       0

The AORC v1.0 data that we're using only covers the Great Basin, whereas `ds_meta` covers the entire CONUS. We'll use the offsets defined in the AORC v1.0 history to subset the `ds_meta` coordinates.

In [12]:
def pattern_lookup(pattern, s):
    
    # use the re.search() function to search for the pattern in the string
    match = re.search(pattern, s)

    # check if a match was found
    if match:
        # extract the matched values and concatenate them into the desired string format
        result = f'{match.group(0)}'
        return result
    else:
        print('No match found.')

In [19]:
# define the regular expression pattern to match the substring
pattern_we = r'west_east,(\d+),(\d+)'
pattern_sn = r'south_north,(\d+),(\d+)'

GSL_westeast = pattern_lookup(pattern_we, ds_aorc.attrs['history'])
GSL_southnorth = pattern_lookup(pattern_sn, ds_aorc.attrs['history'])

y_index = GSL_southnorth.split(',')[1:]
x_index = GSL_westeast.split(',')[1:]

In [26]:
# select the x,y values from ds_meta that correspond with the subset indices in ds_aorc.
# metadata from AORC subsett: west_east,373,1227 -d south_north,1586,2405

leny = len(ds_meta.y)
x = ds_meta.x[int(x_index[0]) : int(x_index[1]) + 1].values
y = ds_meta.y[leny - int(y_index[1]) - 1 : leny - int(y_index[0])].values

Add these values to the AORC v1.0 dataset

In [31]:
# rename the existing dimensions so they are CF compliant
ds_aorc = ds_aorc.rename_dims(south_north='y', west_east='x', Time='time')

In [32]:
# add these x, y values to the AORC dataset
ds_aorc = ds_aorc.assign_coords(y=('south_north', y))
ds_aorc = ds_aorc.assign_coords(x=('west_east', x))
ds_aorc = ds_aorc.set_xindex('y')
ds_aorc = ds_aorc.set_xindex('x')

Add the WRF-Hydro coordinate reference system to AORC v1.0. This `WKT` string can be found within the WRF-Hydro `geo_em.d01_1km.nc` file. 

In [33]:
# add crs to netcdf file
wkt = 'PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DATUM["D_Sphere",SPHEROID["Sphere",6370000.0,0.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",-97.0],PARAMETER["standard_parallel_1",30.0],PARAMETER["standard_parallel_2",60.0],PARAMETER["latitude_of_origin",40.0],UNIT["Meter",1.0]];-35691800 -29075200 10000;-100000 10000;-100000'
ds_aorc.rio.write_crs(wkt, inplace=True);

Add spatial metadata to the `x` and `y` coordinates.

In [35]:

ds_aorc.x.attrs['standard_name'] = "projection_x_coordinate"
ds_aorc.x.attrs['long_name'] = "x coordinate of projection"
ds_aorc.x.attrs['units'] = "m"
ds_aorc.x.attrs['_CoordinateAxisType'] = "GeoX"
ds_aorc.x.attrs['resolution'] = 1000.

ds_aorc.y.attrs['standard_name'] = "projection_y_coordinate"
ds_aorc.y.attrs['long_name'] = "y coordinate of projection"
ds_aorc.y.attrs['units'] = "m"
ds_aorc.y.attrs['_CoordinateAxisType'] = "GeoY"
ds_aorc.y.attrs['resolution'] = 1000.

In [37]:
ds_aorc

<xarray.Dataset>
Dimensions:      (time: 744, y: 820, x: 855, south_north: 820, west_east: 855)
Coordinates:
  * y            (south_north) float64 4.855e+05 4.845e+05 ... -3.335e+05
  * x            (west_east) float64 -1.93e+06 -1.929e+06 ... -1.076e+06
    spatial_ref  int64 0
Dimensions without coordinates: time, south_north, west_east
Data variables:
    LWDOWN       (time, y, x) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    PSFC         (time, y, x) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    Q2D          (time, y, x) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    RAINRATE     (time, y, x) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    SWDOWN       (time, y, x) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    T2D          (time, y, x) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    Times        (time) |S64 dask.array<chunksize=(10,), meta=np.ndarray>
    U2D          (time, y, x) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    V2D          (time, y, x) float32 dask.array<chunksize=(10, 275, 285), meta=np.ndarray>
    valid_time   (time) datetime64[ns] dask.array<chunksize=(10,), meta=np.ndarray>
Attributes:
    history:       Fri Dec 16 20:05:28 2022: ncks /job/data/2010/2010013000.L...
    NCO:           4.7.2
    DODS.strlen:   20
    DODS.dimName:  DateStrLen